In [8]:

from Bio import Entrez
from Bio import SeqIO
import os

Entrez.email = "someone@email.com"
Entrez.tool = "ENTREZ_Retrieval.ipynb"

In [9]:
#find a transcript or gene in nucleotid db and get it from NCBI in genbank format
handle = Entrez.esearch(db="nucleotide", term="NM_002977.3", retmax=100)
record = Entrez.read(handle)
records = []
handle = Entrez.efetch(db="nucleotide", rettype="gbwithparts", retmode="text",
                          id= record['IdList'][0])
records.append(SeqIO.read(handle, 'genbank'))
SeqIO.write(records, f'C:\\Users\\apmcc\\OneDrive\\Documents\\GRT_consulting\\Analysis\\NM_002977.3.gb', "genbank")


{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['256017191'], 'TranslationSet': [], 'QueryTranslation': ''}


In [11]:
#need to make a snapgene gb file with the following additional reference:
# PUBMED   20301342
# REFERENCE   11  (bases 1 to 9771)
# AUTHORS   .
# TITLE     Direct Submission
# JOURNAL   Exported MMM DD, YYYY from SnapGene Viewer 7.1.1
#             https://www.snapgene.com

records = SeqIO.parse(f'NM_002977.3.gb', "genbank")
#snapgene reference
from Bio.SeqFeature import Reference 
ref = Reference()
ref.authors='.'
ref.title='Direct Submission'
ref.journal= 'Exported Jan 10, 2024 from SnapGene Viewer 7.1.1 https://www.snapgene.com'
ref.remark = 'https://www.snapgene.com'
for record in records:
    record.annotations['references'].append(ref)
    for index, feature in enumerate(record.features):
        try:
            if feature.type == "misc_feature":
                #print(feature)
                record.features.pop(index)
        except:
            pass
    SeqIO.write(record, "NM_002977.3.clean.gb", "genbank")

[Reference(title='Structural basis for severe pain caused by mutations in the voltage sensors of sodium channel NaV1.7', ...), Reference(title='Novel SCN9A variant associated with congenital insensitivity to pain', ...), Reference(title='Pain-causing stinging nettle toxins target TMEM233 to modulate NaV1.7 function', ...), Reference(title='An SCN9A channelopathy causes congenital inability to experience pain', ...), Reference(title='Evolution and diversity of mammalian sodium channel genes', ...), Reference(title='A novel tetrodotoxin-sensitive, voltage-gated sodium channel expressed in rat and human dorsal root ganglia', ...), Reference(title='Structure and functional expression of a new member of the tetrodotoxin-sensitive voltage-activated sodium channel family from human neuroendocrine cells', ...), Reference(title='Congenital Insensitivity to Pain Overview', ...), Reference(title='Hereditary Sensory and Autonomic Neuropathy Type II', ...), Reference(title='SCN9A Neuropathic Pain S

In [12]:
#Create a colour list for potency
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np

x = range(0,101,1)
cmap = plt.get_cmap('RdYlGn')(np.linspace(0, 1, len(x)))
color_list = [colors.rgb2hex(c) for c in cmap][::-1]

In [15]:
import pandas as pd

#read sequence
siRNA_seqs = pd.read_csv('all_dfs.csv')
patent_sequences =  siRNA_seqs[['Name','original_as_seqs','Avg_Expression']]
patent_sequences = patent_sequences[patent_sequences['Avg_Expression'] <50]
#     misc_feature    101..121
#     /label=siRNA_check
#     /note="color: #ff0000"
from Bio.Seq import Seq
from Bio import SeqFeature as sf
records = SeqIO.parse(f'NM_002977.3.clean.gb', "genbank")

for record in records:
    for index, patent_sequence in patent_sequences.iterrows():
        sirna_sequence = Seq(patent_sequence['original_as_seqs']).reverse_complement()
        if (record.seq.count(sirna_sequence.upper()) == 0):
            #print(record.seq.count(sirna_sequence))
            print("Sequence not found: ", sirna_sequence)
        else:
            #print("Sequence found")
            val = record.seq.find(sirna_sequence.upper())
            percent_remaining = int(float(patent_sequence['Avg_Expression']))
            if percent_remaining <= 100:
                display_color = color_list[percent_remaining]
            else:
                display_color = color_list[100]
            record.features.append(sf.SeqFeature(sf.FeatureLocation(val,val+len(sirna_sequence)), type="misc_feature", qualifiers= {"label": patent_sequence['Name'], "gene":"SCN9A", "note": f"color: {display_color}", "percent remaining": patent_sequence['Avg_Expression']}))
            #print(record.annotations)
            #.append( {"label": "siRNA1", "gene":"SCN9A", "note": "color: #ff0000"})
        
SeqIO.write(record, "NM_002977.3.clean.anno.50pctKD.gb", "genbank")

Sequence not found:  AAGAATCAAAGAAGCTCTC


1